In [1]:
%pip install llama_index
%pip install llama_index_llms_openai
%pip install llama_index_graph_stores_neo4j
%pip install llama_index_embeddings_openai

Note: you may need to restart the kernel to use updated packages.Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install neo4j

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="neo4j_pass",
    url="bolt://localhost:7687",
)

In [4]:
# Test the connection and verify APOC is available
try:
    # Test basic connection
    print("Testing Neo4j connection...")
    
    # You can also test APOC availability directly
    from neo4j import GraphDatabase
    
    driver = GraphDatabase.driver(
        "bolt://localhost:7687",
        auth=("neo4j", "neo4j_pass")
    )
    
    with driver.session() as session:
        # Test basic connection
        result = session.run("RETURN 'Connection successful' as message")
        print(result.single()["message"])
        
        # Test APOC availability
        try:
            apoc_result = session.run("CALL apoc.help('apoc')")
            print("APOC is available!")
            print(f"Found {len(list(apoc_result))} APOC procedures")
        except Exception as apoc_error:
            print(f"APOC not available: {apoc_error}")
            print("Please install APOC plugin as described above")
    
    driver.close()
    
except Exception as e:
    print(f"Connection failed: {e}")
    print("Make sure Neo4j is running on bolt://localhost:7687")

Testing Neo4j connection...
Connection successful
APOC is available!
Found 436 APOC procedures


In [5]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/").load_data()

In [6]:
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
    kg_extractors=[
        SchemaLLMPathExtractor(
            llm=OpenAI(model="gpt-4o-mini", temperature=0.0)
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

/home/phan.ngoc@sun-asterisk.com/Documents/projects/diary-health/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 11.72it/s]



RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)

nodes = retriever.retrieve("What happened at Interleaf and Viaweb?")

for node in nodes:
    print(node.text)